#### Threat analysis possed by top 50 delegates of ENS

In [4]:
# Cell 1: environment

import matplotlib.font_manager as fm
import numpy as np
import random as rand
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib.pyplot import gca
import os 
import math as mt
from bs4 import BeautifulSoup
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as ticker
import pandas as pd
import requests
import scipy.fftpack
from pprint import pprint
%matplotlib inline

cwd = os.getcwd()
pwd = os.path.abspath(os.path.join(cwd, os.pardir))
fontloc = pwd + '/SF-Mono/SFMono-Regular.otf'
font = fm.FontProperties(fname = fontloc,size = 8); prop = font
proplr = fm.FontProperties(fname = fontloc,size = 12)
prop10 = fm.FontProperties(fname = fontloc,size = 10)

fontlocit = pwd + '/SF-Mono/SFMono-RegularItalic.otf'
fontit = fm.FontProperties(fname = fontlocit,size = 8)

#ticks font
def ticks(ax, size):
    font = fm.FontProperties(fname = fontloc,size = size)
    for label in ax.get_xticklabels():
        label.set_fontproperties(font)
    for label in ax.get_yticklabels():
        label.set_fontproperties(font)
        
def ticks3(ax, x, y, z):
    font = fm.FontProperties(fname = fontloc,size = x)
    for label in ax.get_xticklabels():
        label.set_fontproperties(font)
    font = fm.FontProperties(fname = fontloc,size = y)
    for label in ax.get_yticklabels():
        label.set_fontproperties(font)
    font = fm.FontProperties(fname = fontloc,size = z)
    for label in ax.get_zticklabels():
        label.set_fontproperties(font)
        
def xticks(ax, size):
    font = fm.FontProperties(fname = fontloc,size = size)
    for label in ax.get_xticklabels():
        label.set_fontproperties(font)
        
def yticks(ax, size):
    font = fm.FontProperties(fname = fontloc,size = size)
    for label in ax.get_yticklabels():
        label.set_fontproperties(font)

@ticker.FuncFormatter
def major_formatter(x, pos):
    label = str("{0:.1f}".format(x)) if x < 0 else str("{0:.2f}".format(x))
    return label

import matplotlib.colors as colors

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap


File <code>ENS.log</code> contains voting data of top 50 ENS delegates scrapped from [Sybil.org](https://sybil.org)

In [6]:
# read in data
file_in = cwd + '/ENS.log'
f = open(file_in, 'r')
lines = f.readlines()[1:]
f.close()

rank = []
perc = []
vote = []
name = []

for line in lines:
    if not line.startswith("#"):
        column = line.split()
        rank.append(int(column[0]))
        name.append(str(column[1]))
        perc.append(float(column[2]))
        vote.append(int(column[3]))
            
lines1 = []

In [49]:
%config InlineBackend.figure_format = 'svg'
import itertools

totalVotes = int(np.sum(vote)/(np.sum(perc)*0.01))
print('totalVotes: ' + str(totalVotes))
threshold = 1000000
Total = len(rank)

#N-party attack
N = 5; possVec = []
for perps in range(1, Total):
    print('possVec: ' + str(int(np.math.factorial(Total)/np.math.factorial(Total - perps))))
    possVec.append(int(np.math.factorial(Total)/np.math.factorial(Total - perps)))

for perps in range(3,N + 1):
    print('possVec: ' + str(int(np.math.factorial(Total)/np.math.factorial(Total - perps))))
    perms = list(itertools.permutations(rank,perps))
    threat = []; attackers = []
    for val in perms:
      attack = 0; names = ''; ranks = []
      for i in val:
          attack = attack + vote[int(i - 1)]
          #names = names + name[int(i - 1)] + ';'
          ranks.append(rank[int(i - 1)])
      if attack > threshold:
           #attackers.append(names)
           threat.append(ranks)
      attack = 0

    print(perps, len(threat), len(perms))




totalVotes: 6847327
possVec: 50
possVec: 2450
possVec: 117600
possVec: 5527200
possVec: 254251200
possVec: 11441304000
possVec: 503417376000
possVec: 21646947168000
possVec: 909171781056000
possVec: 37276043023296000
possVec: 1491041720931840000
possVec: 58150627116341755904
possVec: 2209723830420986920960
possVec: 81759781725576512929792
possVec: 2943352142120754599690240
possVec: 103017324974226403472965632
possVec: 3502589049123697992958738432
possVec: 115585438621082022772522090496
possVec: 3698734035874624728720706895872
possVec: 114660755112113382915890562990080
possVec: 3439822653363401235275137756954624
possVec: 99754856947538631787753728827719680
possVec: 2793135994531081763844080702014357504
possVec: 75414671852339211755860851465327214592
possVec: 1960781468160819373426120617748441595904
possVec: 49019536704020484940115925251025627250688
possVec: 1176468880896491715934034661360882235211776
possVec: 27058784260619309156997787389955222685089792
possVec: 595293253733624751936349

NameError: name 'nsmsmk' is not defined

In [47]:
np.math.factorial(50)/np.math.factorial(45)

254251200.0